In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
%load_ext memory_profiler


In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import mne
import keras
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import random
from torch import optim
import argparse
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import torch.nn.functional as F
from os import walk
import matplotlib.pyplot as plt
from copy import deepcopy
import time
from sklearn.preprocessing import MinMaxScaler
from memory_profiler import profile
from sklearn.model_selection import train_test_split

In [3]:
DEVICE = torch.device('cuda:0')

In [4]:
def pad(rawdata, seglength):
    pad_length = -rawdata.size()[0] % seglength
    padded = np.concatenate((rawdata, torch.zeros((int(pad_length), int(rawdata.size()[1])))), 0)
    return padded

In [5]:
def to_epochs(rawdata, seg_length):
    data = pad(rawdata, seg_length)
    split = np.array(np.split(data, seg_length))
    return split.reshape(-1, seg_length, 20)

In [6]:
def to_gpu(x, on_cpu=False, gpu_id=None):

    if torch.cuda.is_available() and not on_cpu:
        x = x.cuda(gpu_id)
    return x

In [7]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(d, device) for d in data]
    return data.to(device, non_blocking=True)

In [8]:
def Data(path, seglength, class_num):
    names = []
    for path, _, name in os.walk(path):
        names.append(name)
        names = np.squeeze(names)
    
    patients = []
    for i in range(len(names)):
        patients.append(torch.Tensor(pd.read_csv(f"{path}\{names[i]}" , skiprows=2, header=None).values))

    patients_epochs = to_epochs(patients[0], seglength)
    for patient in patients:
        patients_epochs = np.concatenate((patients_epochs, to_epochs(patient, seglength)), axis = 0)

    del patients

    
    np.random.shuffle(patients_epochs)
    train = torch.Tensor(patients_epochs)

    del patients_epochs

    train_labels = torch.Tensor(np.ones(len(train)).reshape(-1) * class_num)
    
    train_labels = torch.zeros([len(train_labels), 4])
    train_labels[:, class_num - 1] = 1

    train = deepcopy(train.view(-1, seglength, 20)[:, :, 2:])

    train = deepcopy((train - torch.min(train))/(torch.max(train) - torch.min(train))*(2) + -1)
    return train, train_labels

In [10]:
if __name__ == "__main__":
    seglength = 9000
    train_ratio = 0.8

    #focal_train, focal_test, focal_train_labels, focal_test_labels = Data('D:\EEG\FOCAL', train_ratio, seglength, class_num = 1)
    #ige_train, ige_test, ige_train_labels, ige_test_labels = Data('D:\EEG\IGE', train_ratio, seglength, class_num = 2)
    #pnes_train, pnes_test, pnes_train_labels, pnes_test_labels = Data('D:\EEG\PNES', train_ratio, seglength, class_num = 3)
    #temporal, temporal_labels = Data('D:\EEG\TLE', seglength, class_num = 4)

    """
    print(f"train shape: {temporal_train.shape}")
    print(f"test shape: {temporal_test.shape}")
    print(f"train labels shape: {temporal_train_labels.shape}")
    print(f"test labels shape: {temporal_test_labels.shape}")
    """


In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(temporal, temporal_labels, test_size=0.25, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)